In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import warnings
import scipy as sp
import time, copy, pickle, matplotlib, os, sys, math, random, itertools
import matplotlib.pyplot as plt
import networkx as nx

import torch_geometric as tg
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv, GCNConv, GATv2Conv
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax, get_laplacian, to_dense_adj

from my_gat import my_GATConv
from my_mlp_gat import GATv3Layer, OGGATLayer

np.seterr(all="ignore")

sys.path.insert(0, os.path.abspath('../../'))

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class Model_GCN(torch.nn.Module):
    def __init__(self, d, out_d):
        super(Model_GCN, self).__init__()
        
        self.conv1 = GCNConv(d,out_d, bias=False)

    def forward(self, data):
        x = data.x
        x = self.conv1(x, data.edge_index)
        
        return x.squeeze(-1)
    
class Model_linear(torch.nn.Module):
    def __init__(self, d, out_d):
        super(Model_linear, self).__init__()
        
        self.linear = torch.nn.Linear(d,out_d, bias=False)

    def forward(self, data):
        x = data.x
        x = self.linear(x)
        
        return x.squeeze(-1)

In [ ]:
class Eigen2(nn.Module):
    def __init__(self, k):
        super().__init__()
        self.k = k
        
    def forward(self, edge_idx, n, sigma):
        # lap_idx, lap_wt = get_laplacian(edge_idx, normalization="sym")
        edge_adj = to_dense_adj(edge_idx)
        eigenvals, eigenvecs = torch.linalg.eig(edge_adj)
        top_eig = eigenvecs.squeeze(0)[:, 1:self.k+1]
        top_eig = torch.real(top_eig)
        
        new_edge_features = torch.Tensor(edge_idx.size(1), 1).to(edge_idx.device)
        new_edge_idx = edge_idx.T
        
        for idx, pair in enumerate(new_edge_idx):
            i, j = pair
            x_i_prime = top_eig[i]
            x_j_prime = top_eig[j]
            dot = torch.dot(x_i_prime, x_j_prime)
            final = 8 * torch.sqrt(torch.log(torch.tensor(n))) * sigma * torch.sign(dot)
            new_edge_features[idx] = final
            
        return new_edge_features.view(-1, 1), eigenvals, top_eig

class Model_GATv3(torch.nn.Module):
    def __init__(self, d, out_d):
        super(Model_GATv3, self).__init__()
        
        self.eigen = Eigen2(1)
        self.conv1 = GATv3Layer(
            in_channels=d, 
            out_channels=out_d,
            att_in_channels=2,
            att_out_channels=4,
            share_weights=True, 
            bias=False
        )

    def forward(self, data, mu):
        x = data.x
        n = x.size(0)
        sigma = torch.std(x)
        eigen_x, _, _ = self.eigen(data.edge_index, n=n, sigma=sigma)
        x, attn_weights, pair_pred = self.conv1(x, edge_index=data.edge_index, edge_attr=eigen_x, cur_mu=mu, return_attention_info=True)
        
        return x.squeeze(-1), attn_weights, pair_pred

In [ ]:
@torch.no_grad()
def measure_accuracy(model, data):
    model.eval()
         
    logits = model(data) # forward operation
    preds = torch.sigmoid(logits) > 0.5
    
    # calculate training accuracy
    correct = preds[data.train_mask] == data.ynew[data.train_mask]
    train_acc = int(correct.sum()) / int(data.train_mask.sum())
    
    # calculate training accuracy
    correct = preds[data.test_mask] == data.ynew[data.test_mask]
    test_acc = int(correct.sum()) / int(data.test_mask.sum())
        
    return train_acc, test_acc

@torch.no_grad()
def measure_accuracy_gat(model, data, mu):
    model.eval()
         
    logits, attn_weights, pair_pred = model(data, mu) # forward operation
    preds = torch.sigmoid(logits) > 0.5
    
    # calculate training accuracy
    correct = preds[data.train_mask] == data.ynew[data.train_mask]
    train_acc = int(correct.sum()) / int(data.train_mask.sum())
    
    # calculate training accuracy
    correct = preds[data.test_mask] == data.ynew[data.test_mask]
    test_acc = int(correct.sum()) / int(data.test_mask.sum())
        
    return train_acc, test_acc, attn_weights, pair_pred

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def info_mlp_gat(attn_weights, idx, head):
    intra_weight = []
    idx_intra = []
    inter_weight = []
    idx_inter = []

    weights = attn_weights[1] # get gamma values

    idx_class0 = idx.nonzero(as_tuple=True)[0]
    idx_class1 = (~idx).nonzero(as_tuple=True)[0]
    
    for i in range(attn_weights[0].shape[1]):
        edge = [attn_weights[0][0][i].item(),attn_weights[0][1][i].item()]
        if ((edge[0] in idx_class0) and (edge[1] in idx_class0)) or ((edge[0] in idx_class1) and (edge[1] in idx_class1)):
            intra_weight.append(weights[i].item())
            idx_intra.append(i)
        else:
            inter_weight.append(weights[i].item())
            idx_inter.append(i)
            
    return intra_weight, idx_intra, inter_weight, idx_inter

def info_mlp_gat_scores_phi(attn_weights, idx, head):
    idx_intra = []
    idx_inter = []
    
    intra_weight_00 = []
    intra_weight_11 = []
    
    inter_weight_01 = []
    inter_weight_10 = []

    weights = attn_weights[2] # get attention scores

    idx_class0 = idx.nonzero(as_tuple=True)[0]
    idx_class1 = (~idx).nonzero(as_tuple=True)[0]
    
    for i in range(attn_weights[0].shape[1]):
        edge = [attn_weights[0][0][i].item(),attn_weights[0][1][i].item()]
        if ((edge[0] in idx_class0) and (edge[1] in idx_class0)):
            intra_weight_00.append(weights[i].item())
            idx_intra.append(i)
        elif ((edge[0] in idx_class1) and (edge[1] in idx_class1)):
            intra_weight_11.append(weights[i].item())
            idx_intra.append(i)
        elif ((edge[0] in idx_class0) and (edge[1] in idx_class1)):
            inter_weight_01.append(weights[i].item())
            idx_inter.append(i)
        elif ((edge[0] in idx_class1) and (edge[1] in idx_class0)):
            inter_weight_10.append(weights[i].item())
            idx_inter.append(i)
            
    return intra_weight_00, intra_weight_11, idx_intra, inter_weight_01, inter_weight_10, idx_inter

def info_mlp_gat_scores_psi(attn_weights, idx, head):
    idx_intra = []
    idx_inter = []
    
    intra_weight_00 = []
    intra_weight_11 = []
    
    inter_weight_01 = []
    inter_weight_10 = []

    weights = attn_weights[3] # get attention scores

    idx_class0 = idx.nonzero(as_tuple=True)[0]
    idx_class1 = (~idx).nonzero(as_tuple=True)[0]
    
    for i in range(attn_weights[0].shape[1]):
        edge = [attn_weights[0][0][i].item(),attn_weights[0][1][i].item()]
        if ((edge[0] in idx_class0) and (edge[1] in idx_class0)):
            intra_weight_00.append(weights[i].item())
            idx_intra.append(i)
        elif ((edge[0] in idx_class1) and (edge[1] in idx_class1)):
            intra_weight_11.append(weights[i].item())
            idx_intra.append(i)
        elif ((edge[0] in idx_class0) and (edge[1] in idx_class1)):
            inter_weight_01.append(weights[i].item())
            idx_inter.append(i)
        elif ((edge[0] in idx_class1) and (edge[1] in idx_class0)):
            inter_weight_10.append(weights[i].item())
            idx_inter.append(i)
            
    return intra_weight_00, intra_weight_11, idx_intra, inter_weight_01, inter_weight_10, idx_inter

def pair_acc(n_edges, pair_pred, idx_intra, idx_inter, head):
    
    tmp = torch.zeros(n_edges).to(device)
    tmp[pair_pred[:,head].reshape(len(pair_pred)) > 0] = 1

    gt = torch.zeros(n_edges).to(device)
    gt[idx_intra] = 1

    acc_intra_edges = 1 - torch.sum(torch.abs(gt[idx_intra] - tmp[idx_intra]))/len(idx_intra)
    acc_inter_edges = 1 - torch.sum(torch.abs(gt[idx_inter] - tmp[idx_inter]))/len(idx_inter)
    print("\tHead: ", head, " acc intra edges: ", acc_intra_edges.item(), " acc inter edges: ", acc_inter_edges.item())
    
    return acc_intra_edges, acc_inter_edges

In [ ]:
def my_plot(name, which_class, heads, allqs, degree, intra_gamma, inter_gamma, intra_gamma_, inter_gamma_, 
            intra_gamma_std, inter_gamma_std, intra_gamma_std_, inter_gamma_std_, 
            test_acc_mlp_gat, test_acc_gat, test_acc_gcn, test_acc_lin, 
            acc_intra_edges_all, acc_inter_edges_all, acc_intra_edges_all_, acc_inter_edges_all_,
            phi_intra_attn_00, phi_intra_attn_11, phi_inter_attn_01, phi_inter_attn_10,
            phi_intra_attn_00_std, phi_intra_attn_11_std, phi_inter_attn_01_std, phi_inter_attn_10_std,
            psi_intra_attn_00, psi_intra_attn_11, psi_inter_attn_01, psi_inter_attn_10,
            psi_intra_attn_00_std, psi_intra_attn_11_std, psi_inter_attn_01_std, psi_inter_attn_10_std, 
            indic_data, parent_path="final_q_plots"):
    
    allqs = allqs.cpu()
    
    fig = plt.figure(figsize=(11, 7), dpi=80)

    plt.plot(allqs,intra_gamma, linewidth=2, linestyle='-', marker='*', markersize=9, label='Average $\gamma$, intra edges, MLP-GAT')
    # plt.fill_sbetween(allqs,np.asarray(intra_gamma)-np.asarray(intra_gamma_std),np.asarray(intra_gamma)+np.asarray(intra_gamma_std),alpha=0.4)

    plt.plot(allqs,inter_gamma, linewidth=2, linestyle='-', marker='X', markersize=9, label='Average $\gamma$, inter edges, MLP-GAT')
    # plt.fill_between(allqs,np.asarray(inter_gamma)-np.asarray(inter_gamma_std),np.asarray(inter_gamma)+np.asarray(inter_gamma_std),alpha=0.4)
    
    d_mean = (1+degree).mean().cpu()
#     inv_d_std = (1/(1+degree)).std().cpu()
    inv_degree_mean_vec = (1/d_mean)*np.ones(len(allqs))
    
    plt.plot(allqs,inv_degree_mean_vec, linewidth=2, linestyle='-', marker='o', markersize=9, color = 'black', label='Average $1/|N_i|$')
#     plt.fill_between(allqs,inv_degree_mean_vec-inv_d_std,inv_degree_mean_vec+inv_d_std,alpha=0.4)

    plt.grid(linestyle='dashed')
    plt.legend(fontsize=20)
    plt.xscale('log')
    plt.tick_params(axis='x', labelsize=18)
    plt.tick_params(axis='y', labelsize=18)
    plt.xlabel('$q$ Value', fontsize=20)
    plt.ylabel('$\gamma$ value', fontsize=20)
    plt.show()
    
    fig.savefig(parent_path + "/gammas_MLPGAT_real_data_ansatz_"+name+"_class_"+str(which_class)+"_indicator.pdf", dpi=400, bbox_inches='tight')
    
    fig = plt.figure(figsize=(11, 7), dpi=80)

    plt.plot(allqs, intra_gamma_std, linewidth=2, linestyle='-', marker='*', markersize=9, label='Stand. dev. $\gamma$, intra edges, MLP-GAT')

    plt.plot(allqs, inter_gamma_std, linewidth=2, linestyle='-', marker='X', markersize=9, label='Stand. dev. $\gamma$, inter edges, MLP-GAT')
    
#     d_mean = (1+degree).mean().cpu()
    inv_d_std = (1/(1+degree)).std().cpu()
#     inv_degree_mean_vec = (1/d_mean)*np.ones(len(allqs))
    
    plt.plot(allqs,inv_d_std*np.ones(len(allqs)), linewidth=2, linestyle='-', marker='o', markersize=9, color = 'black', label='Stand. dev. of $1/|N_i|$')

    plt.grid(linestyle='dashed')
    plt.legend(fontsize=20)
    plt.xscale('log')
    plt.tick_params(axis='x', labelsize=18)
    plt.tick_params(axis='y', labelsize=18)
    plt.xlabel('$q$ Value', fontsize=20)
    plt.ylabel('Value', fontsize=20)
    plt.show()
    
    fig.savefig(parent_path + "/gammas_std_MLPGAT_real_data_ansatz_"+name+"_class_"+str(which_class)+"_indicator.pdf", dpi=400, bbox_inches='tight')
    
    fig = plt.figure(figsize=(16, 12), dpi=80)

    marker_intra = ['v','^','>','<']
    marker_inter = ['1','2','3','4']
    
    fig = plt.figure(figsize=(11, 7), dpi=80)

    plt.plot(allqs,test_acc_mlp_gat, linewidth=2, linestyle='-', marker='*', markersize=9, label='MLP-GAT')
    plt.plot(allqs,test_acc_gcn, linewidth=2, linestyle='-', marker='s', markersize=9, label='GCN')
    plt.plot(allqs,test_acc_lin, linewidth=2, linestyle='-', marker='D', markersize=9, color = 'purple', label='No-graph')
    
    plt.axvline(x=0.0008, color='green', linestyle='--', label='q=0.0008')

    plt.grid(linestyle='dashed')
    plt.legend(fontsize=20)
    plt.xscale('log')
    plt.tick_params(axis='x', labelsize=18)
    plt.tick_params(axis='y', labelsize=18)
    plt.xlabel('$q$ Value', fontsize=20)
    plt.ylabel('Classification accuracy', fontsize=20)
    plt.show()
    
    fig.savefig(parent_path + "/node_classification_MLPGAT_real_data_ansatz_"+name+"_class_"+str(which_class)+"_indicator.pdf", dpi=400, bbox_inches='tight')
    
    fig = plt.figure(figsize=(11, 7), dpi=80)

    plt.plot(allqs,acc_intra_edges_all, linewidth=2, linestyle='-', marker='*', markersize=9, label='MLP-GAT, intra edge classification')
    plt.plot(allqs,acc_inter_edges_all, linewidth=2, linestyle='-', marker='X', markersize=9, label='MLP-GAT, inter edge classification')

    plt.axvline(x=0.0008, color='green', linestyle='--', label='q=0.0008')

    plt.grid(linestyle='dashed')
    plt.legend(fontsize=20)
    plt.xscale('log')
    plt.tick_params(axis='x', labelsize=18)
    plt.tick_params(axis='y', labelsize=18)
    plt.xlabel('$q$ Value', fontsize=20)
    plt.ylabel('Classification accuracy', fontsize=20)
    plt.show()
    
    fig.savefig(parent_path + "/edge_classification_MLPGAT_real_data_ansatz_"+name+"_class_"+str(which_class)+"_indicator.pdf", dpi=400, bbox_inches='tight')
    
    # ---------------------------------------PSI---------------------------------------
    
#     fig = plt.figure(figsize=(11, 7), dpi=80)
    
#     plt.plot(allqs, psi_intra_attn_00, label="$i \in C_0, j \in C_0$", linewidth=2, linestyle="-", markersize=9, marker="*")
#     plt.fill_between(allqs, np.asarray(psi_intra_attn_00)-np.asarray(psi_intra_attn_00_std), np.asarray(psi_intra_attn_00)+np.asarray(psi_intra_attn_00_std), alpha=0.4)
    
#     plt.plot(allqs, psi_intra_attn_11, label="$i \in C_1, j \in C_1$", linewidth=2, linestyle="-", markersize=9, marker="X")
#     plt.fill_between(allqs, np.asarray(psi_intra_attn_11)-np.asarray(psi_intra_attn_11_std), np.asarray(psi_intra_attn_11)+np.asarray(psi_intra_attn_11_std), alpha=0.4)
    
#     plt.plot(allqs, psi_inter_attn_10, label="$i \in C_1, j \in C_0$", linewidth=2, linestyle="-", markersize=9, marker="+")
#     plt.fill_between(allqs, np.asarray(psi_inter_attn_10)-np.asarray(psi_inter_attn_10_std), np.asarray(psi_inter_attn_10)+np.asarray(psi_inter_attn_10_std), alpha=0.4)
    
#     plt.plot(allqs, psi_inter_attn_01, label="$i \in C_0, j \in C_1$", linewidth=2, linestyle="-", markersize=9, marker="o")
#     plt.fill_between(allqs, np.asarray(psi_inter_attn_01)-np.asarray(psi_inter_attn_01_std), np.asarray(psi_inter_attn_01)+np.asarray(psi_inter_attn_01_std), alpha=0.4)
    
#     plt.grid(linestyle="dashed")
#     plt.legend(fontsize=20)
#     plt.xscale('log')
#     plt.tick_params(axis='x', labelsize=18)
#     plt.tick_params(axis='y', labelsize=18)
#     plt.xlabel('$q$ Value', fontsize=20)
#     plt.ylabel('$\Psi$ Values', fontsize=20)
#     plt.show()
    
#     fig.savefig(parent_path + "/psi_values_MLPGAT_real_data_ansatz_"+name+"_class_"+str(which_class)+"_indicator.pdf", dpi=400, bbox_inches='tight')
    
#     # ---------------------------------------PHI---------------------------------------
    
#     fig = plt.figure(figsize=(11, 7), dpi=80)
    
#     plt.plot(allqs, phi_intra_attn_00, label="$i \in C_0, j \in C_0$", linewidth=2, linestyle="-", markersize=9, marker="*")
#     plt.fill_between(allqs, np.asarray(phi_intra_attn_00)-np.asarray(phi_intra_attn_00_std), np.asarray(phi_intra_attn_00)+np.asarray(phi_intra_attn_00_std), alpha=0.4)
    
#     plt.plot(allqs, phi_intra_attn_11, label="$i \in C_1, j \in C_1$", linewidth=2, linestyle="-", markersize=9, marker="X")
#     plt.fill_between(allqs, np.asarray(phi_intra_attn_11)-np.asarray(phi_intra_attn_11_std), np.asarray(phi_intra_attn_11)+np.asarray(phi_intra_attn_11_std), alpha=0.4)
    
#     plt.plot(allqs, phi_inter_attn_10, label="$i \in C_1, j \in C_0$", linewidth=2, linestyle="-", markersize=9, marker="+")
#     plt.fill_between(allqs, np.asarray(phi_inter_attn_10)-np.asarray(phi_inter_attn_10_std), np.asarray(phi_inter_attn_10)+np.asarray(phi_inter_attn_10_std), alpha=0.4)
    
#     plt.plot(allqs, phi_inter_attn_01, label="$i \in C_0, j \in C_1$", linewidth=2, linestyle="-", markersize=9, marker="o")
#     plt.fill_between(allqs, np.asarray(phi_inter_attn_01)-np.asarray(phi_inter_attn_01_std), np.asarray(phi_inter_attn_01)+np.asarray(phi_inter_attn_01_std), alpha=0.4)
    
#     plt.grid(linestyle="dashed")
#     plt.legend(fontsize=20)
#     plt.xscale('log')
#     plt.tick_params(axis='x', labelsize=18)
#     plt.tick_params(axis='y', labelsize=18)
#     plt.xlabel('Distance between means', fontsize=20)
#     plt.ylabel('$\Phi$ Values', fontsize=20)
#     plt.show()
    
#     fig.savefig(parent_path + "/phi_values_MLPGAT_real_data_ansatz_"+name+"_class_"+str(which_class)+"_indicator.pdf", dpi=400, bbox_inches='tight')

In [ ]:
def create_ansatz_mlp_gat(model, mean, R):
    
    model_mlp_gat_ansatz = copy.deepcopy(model)
    w = (R/torch.norm(mean))*mean
    
    with torch.no_grad():
        model_mlp_gat_ansatz.conv1.lin.weight = torch.nn.Parameter(w.reshape(1,w.shape[0]))
        model_mlp_gat_ansatz.conv1.att_in.weight = torch.nn.Parameter(torch.tensor([[1.0,1.0],[-1.0,-1.0],[1.0,-1.0],[-1.0,1.0]])) # S
        model_mlp_gat_ansatz.conv1.att_out.weight = torch.nn.Parameter(torch.tensor([[1.0, 1.0, -1.0, -1.0]])) # r

    model_mlp_gat_ansatz.to(device)

    return model_mlp_gat_ansatz

def create_ansatz_gcn(model, mean, R):
    
    model_gcn_ansatz = copy.deepcopy(model)
    w = (R/torch.norm(mean))*mean
    
    with torch.no_grad():
        model_gcn_ansatz.conv1.lin.weight = torch.nn.Parameter(w.reshape(1,w.shape[0]))

    model_gcn_ansatz.to(device)

    return model_gcn_ansatz

def create_ansatz_linear(model, mean, R):
    
    model_linear_ansatz = copy.deepcopy(model)
    w = (R/torch.norm(mean))*mean
    
    with torch.no_grad():
        model_linear_ansatz.linear.weight = torch.nn.Parameter(w.reshape(1,w.shape[0]))

    model_linear_ansatz.to(device)

    return model_linear_ansatz

In [ ]:
def get_graph_stats(edge_idx, y, x0, x1):
    assert edge_idx.size(0) == 2
    new_edge_idx = edge_idx.T
    np = 0
    nq = 0
    
    for idx, pair in enumerate(new_edge_idx):
        i, j = pair
        if y[i] == y[j]:
            np += 1
        else:
            nq += 1
    
    total_intra = math.comb(x0.size(0), 2) + math.comb(x1.size(0), 2)
    total_inter = (x0.size(0) * x1.size(0))
    
    p = np/total_intra
    q = nq/total_inter
        
    return p, q, np, nq

In [ ]:
def get_new_edge_idx(artq, data, x0, x1, x_final, y_final):
    nc0 = x0.size(0)
    nc1 = x1.size(0)
    
    cur_edge_idx = data.edge_index
    N_inter = int(artq * (nc0 * nc1))
    
    inter_edges = []
    inter_edges_count = 0
    
    only_intra = []
    for idx, pair in enumerate(cur_edge_idx.T):
        i, j = pair
        if y_final[i] == y_final[j]:
            only_intra.append(
                        [int(i.item()), int(j.item())]
                    )
            
    only_intra = torch.tensor(np.asarray(only_intra))

    while inter_edges_count < N_inter:
        randi, randj = random.randint(0, x_final.size(0)-1), random.randint(0, x_final.size(0)-1)
        if randi != randj and y_final[randi] != y_final[randj]:
            inter_edges.append([int(randi), int(randj)])
            inter_edges_count += 1

    inter_edges = torch.from_numpy(np.asarray(inter_edges))
    
    new_edge_idx = torch.cat([only_intra, inter_edges], dim=0).T.long()
    assert new_edge_idx.size(0) == 2
    
    return new_edge_idx

In [ ]:
datasets = [Planetoid(root='data/CiteSeer/', name='CiteSeer'), Planetoid(root='data/Cora/', name='Cora'), Planetoid(root='data/PubMed/', name='PubMed')]

for dataset in datasets:

    data = dataset[0].to(device)
    n_classes = data.y.max() + 1
    n = data.y.shape[0]
    data.x_backup = copy.deepcopy(data.x) 
    degree = tg.utils.degree(data.edge_index[1], n)
    
    count = 0

    for which_class in range(n_classes):

        c0 = which_class

        y_final = torch.zeros(n, dtype=torch.float64)
        idx = data.y == c0
        y_final[idx] = 1
        data.ynew = y_final

        x0 = data.x[data.y == c0]
        x1 = data.x[data.y != c0]
        x_final = torch.cat([x0, x1])

        mean0 = data.x_backup[idx].mean(dim=0)
        mean1 = data.x_backup[~idx].mean(dim=0)

        distance = torch.norm(mean0-mean1)
        mu_ = distance / (2*torch.sqrt(torch.tensor(data.x.size(1))))

        # mu_up = 10*mu_
        # mu_lb = 0.01*mu_

        p, q, n_intra, n_inter = get_graph_stats(data.edge_index, y_final, x0, x1)
        print ("distance:", distance)
        print ("class:", which_class)
        print ("effective p:", p)
        print ("effective q:", q)
        print ("# intra:", n_intra)
        print ("# inter:", n_inter)
        print ("--------------------------------------------------------\n")

        all_qs = torch.tensor(np.linspace(0, p, num=30, endpoint=True))

        test_acc_mlp_gat = []
        test_acc_gat = []
        test_acc_gcn = []
        test_acc_lin = []
        intra_gamma = []
        inter_gamma = []
        intra_gamma_std = []
        inter_gamma_std = []
        acc_intra_edges_all = []
        acc_inter_edges_all = []
        intra_gamma_ = []
        inter_gamma_ = []
        intra_gamma_std_ = []
        inter_gamma_std_ = []
        acc_intra_edges_all_ = []
        acc_inter_edges_all_ = []

        phi_intra_attn_00 = []
        phi_intra_attn_11 = []        
        phi_inter_attn_01 = []
        phi_inter_attn_10 = []

        phi_intra_attn_00_std = []
        phi_intra_attn_11_std = []        
        phi_inter_attn_01_std = []
        phi_inter_attn_10_std = []

        psi_intra_attn_00 = []
        psi_intra_attn_11 = []        
        psi_inter_attn_01 = []
        psi_inter_attn_10 = []

        psi_intra_attn_00_std = []
        psi_intra_attn_11_std = []        
        psi_inter_attn_01_std = []
        psi_inter_attn_10_std = []

        heads = 2
        for head in range(heads):
            intra_gamma_.append([[],[]])
            inter_gamma_.append([[],[]])
            intra_gamma_std_.append([[],[]])
            inter_gamma_std_.append([[],[]])
            acc_intra_edges_all_.append([])
            acc_inter_edges_all_.append([])

        original_edge_idx = data.edge_index
        
        indic_data = []

        for q_idx, q in enumerate(all_qs):
            print (f"idx: {q_idx}/{len(all_qs)} | current mu: {q}")
            
            new_mean0 = mu_*torch.ones(data.x.shape[1]).to(device)
            new_mean1 = -mu_*torch.ones(data.x.shape[1]).to(device)
            
            mu_tilda_diff = new_mean0 - new_mean1

            data.x[idx] = data.x_backup[idx] - mean0 + new_mean0
            data.x[~idx] = data.x_backup[~idx] - mean1 + new_mean1

            # data.edge_index = original_edge_idx
            # new_edge_idx = get_new_edge_idx(q, data, x0, x1, x_final, y_final)
            # data.edge_index = new_edge_idx

            model_gcn_ansatz = create_ansatz_gcn(Model_GCN(data.x.shape[1], out_d=1).to(device), new_mean0, 1)
            train_acc, test_acc = measure_accuracy(model_gcn_ansatz, data)
            print(f"GCN\t\tTrain: {train_acc:0.4f} | Test: {test_acc:0.4f}")
            test_acc_gcn.append(test_acc)

            model_linear_ansatz = create_ansatz_linear(Model_linear(data.x.shape[1], out_d=1).to(device), new_mean0, 1)
            train_acc, test_acc = measure_accuracy(model_linear_ansatz, data)
            print(f"Linear\t\tTrain: {train_acc:0.4f} | Test: {test_acc:0.4f}")
            test_acc_lin.append(test_acc)

            model_gatv3_ansatz = create_ansatz_mlp_gat(Model_GATv3(data.x.shape[1], out_d=1).to(device), new_mean0, 1)
            train_acc, test_acc, attn_weights, pair_pred = measure_accuracy_gat(model_gatv3_ansatz, data, mu_tilda_diff)
            print(f"MLP GAT ansatz\t\tTrain: {train_acc:0.4f} | Test: {test_acc:0.4f}")
            test_acc_mlp_gat.append(test_acc)

            intra_weight, idx_intra, inter_weight, idx_inter = info_mlp_gat(attn_weights, idx, 0)

            intra_gamma.append(np.asarray(intra_weight).mean())
            inter_gamma.append(np.asarray(inter_weight).mean())
            intra_gamma_std.append(np.asarray(intra_weight).std())
            inter_gamma_std.append(np.asarray(inter_weight).std())
            acc_intra_edges, acc_inter_edges = pair_acc(attn_weights[0][0].shape[0], pair_pred, idx_intra, idx_inter, 0)
            acc_intra_edges_all.append(acc_intra_edges.cpu())
            acc_inter_edges_all.append(acc_inter_edges.cpu())

            phi_intra_weight_00, phi_intra_weight_11, phi_idx_intra_scores, phi_inter_weight_01, phi_inter_weight_10, phi_idx_inter_scores = info_mlp_gat_scores_phi(attn_weights, idx, 0) # analyse phi scores
            # psi_intra_weight_00, psi_intra_weight_11, psi_idx_intra_scores, psi_inter_weight_01, psi_inter_weight_10, psi_idx_inter_scores = info_mlp_gat_scores_psi(attn_weights, idx, 0) # analyse psi scores
            
            def indicator_fn(n, sigma, mu):
                if np.linalg.norm(mu) >= 1 * sigma * np.sqrt(2 * np.log(n)):
                    return 1
                else:
                    return 0

            current_sigma = torch.std(data.x)
            current_indic = indicator_fn(data.x.size(0), current_sigma, mu_tilda_diff)
            print ("norm:", np.linalg.norm(mu_tilda_diff), "threshold:", 1 * current_sigma * np.sqrt(2 * np.log(data.x.size(0))))

            if current_indic == 1:
                print ("THRESHOLD PASSED")
                count += 1
                
            phi_intra_attn_00.append(np.asarray(phi_intra_weight_00).mean())
            phi_intra_attn_11.append(np.asarray(phi_intra_weight_11).mean())
            phi_inter_attn_01.append(np.asarray(phi_inter_weight_01).mean())
            phi_inter_attn_10.append(np.asarray(phi_inter_weight_10).mean())

            phi_intra_attn_00_std.append(np.asarray(phi_intra_weight_00).std())
            phi_intra_attn_11_std.append(np.asarray(phi_intra_weight_11).std())
            phi_inter_attn_01_std.append(np.asarray(phi_inter_weight_01).std())
            phi_inter_attn_10_std.append(np.asarray(phi_inter_weight_10).std())

#             psi_intra_attn_00.append(np.asarray(psi_intra_weight_00).mean())
#             psi_intra_attn_11.append(np.asarray(psi_intra_weight_11).mean())
#             psi_inter_attn_01.append(np.asarray(psi_inter_weight_01).mean())
#             psi_inter_attn_10.append(np.asarray(psi_inter_weight_10).mean())

#             psi_intra_attn_00_std.append(np.asarray(psi_intra_weight_00).std())
#             psi_intra_attn_11_std.append(np.asarray(psi_intra_weight_11).std())
#             psi_inter_attn_01_std.append(np.asarray(psi_inter_weight_01).std())
#             psi_inter_attn_10_std.append(np.asarray(psi_inter_weight_10).std())

            indic_data.append(
                [np.linalg.norm(mu_tilda_diff), current_sigma, 1 * current_sigma * np.sqrt(2 * np.log(data.x.size(0))), current_indic, data.x.size(0)]
            )
            
        print ("Indic len:", len(indic_data))

        my_plot(dataset.name, which_class, heads, all_qs, degree, intra_gamma, inter_gamma, intra_gamma_, inter_gamma_, 
                intra_gamma_std, inter_gamma_std, intra_gamma_std_, inter_gamma_std_, 
                test_acc_mlp_gat, test_acc_gat, test_acc_gcn, test_acc_lin, 
                acc_intra_edges_all, acc_inter_edges_all, acc_intra_edges_all_, acc_inter_edges_all_,
                phi_intra_attn_00, phi_intra_attn_11, phi_inter_attn_01, phi_inter_attn_10,
                phi_intra_attn_00_std, phi_intra_attn_11_std, phi_inter_attn_01_std, phi_inter_attn_10_std,
                psi_intra_attn_00, psi_intra_attn_11, psi_inter_attn_01, psi_inter_attn_10,
                psi_intra_attn_00_std, psi_intra_attn_11_std, psi_inter_attn_01_std, psi_inter_attn_10_std, 
                indic_data, parent_path="real_data_indic_coeff_1_placebo")

        print ()
    print ("Pass count:", count, "\n\n")